In [2]:
from pystac_client import Client
import geopandas as gpd

In [3]:
api_url = "https://earth-search.aws.element84.com/v0"
toms_run = gpd.read_file("TomsRun.geojson")
toms_bbox = toms_run.total_bounds
collection = "sentinel-s2-l2a-cogs"

In [4]:
client = Client.open(api_url)

In [5]:
search = client.search(
    collections=[collection],
    bbox=toms_bbox,
    datetime=['2023-01-01', '2023-03-20'],
    max_items=10,
    query={"eo:cloud_cover": {"lt": 20, "gte":0}}
)
print(search.matched())

5


In [6]:
items = search.get_all_items()
print(len(items))

5


In [7]:
for item in items:
    print(item)

<Item id=S2B_17SND_20230317_0_L2A>
<Item id=S2B_17SND_20230307_0_L2A>
<Item id=S2B_17SND_20230304_0_L2A>
<Item id=S2A_17SND_20230210_0_L2A>
<Item id=S2A_17SND_20230131_0_L2A>


In [8]:
item = items[0]
print(f"First images:\n Datetime: {item.datetime}\n Geometry: {item.geometry}\n Properties: {item.properties}")

First images:
 Datetime: 2023-03-17 16:22:31+00:00
 Geometry: {'type': 'Polygon', 'coordinates': [[[-79.73660812786935, 38.75404518170041], [-81.00021868625561, 38.76087346847721], [-81.0002217816295, 39.7502589163948], [-79.71873148679177, 39.743187312549146], [-79.73660812786935, 38.75404518170041]]]}
 Properties: {'datetime': '2023-03-17T16:22:31Z', 'platform': 'sentinel-2b', 'constellation': 'sentinel-2', 'instruments': ['msi'], 'gsd': 10, 'view:off_nadir': 0, 'proj:epsg': 32617, 'sentinel:utm_zone': 17, 'sentinel:latitude_band': 'S', 'sentinel:grid_square': 'ND', 'sentinel:sequence': '0', 'sentinel:product_id': 'S2B_MSIL2A_20230317T160919_N0509_R140_T17SND_20230317T204350', 'sentinel:data_coverage': 100, 'eo:cloud_cover': 0, 'sentinel:valid_cloud_cover': False, 'sentinel:processing_baseline': '05.09', 'sentinel:boa_offset_applied': True, 'created': '2023-03-17T23:50:35.241Z', 'updated': '2023-03-17T23:50:35.241Z'}


## Accessing the assets

In [9]:
assets = items[0].assets
print(assets.keys())

dict_keys(['thumbnail', 'overview', 'info', 'metadata', 'visual', 'B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12', 'AOT', 'WVP', 'SCL'])


In [10]:
for key, asset in assets.items():
    print(f"{key}: {asset.title}")

thumbnail: Thumbnail
overview: True color image
info: Original JSON metadata
metadata: Original XML metadata
visual: True color image
B01: Band 1 (coastal)
B02: Band 2 (blue)
B03: Band 3 (green)
B04: Band 4 (red)
B05: Band 5
B06: Band 6
B07: Band 7
B08: Band 8 (nir)
B8A: Band 8A
B09: Band 9
B11: Band 11 (swir16)
B12: Band 12 (swir22)
AOT: Aerosol Optical Thickness (AOT)
WVP: Water Vapour (WVP)
SCL: Scene Classification Map (SCL)


In [11]:
print(assets["thumbnail"].href)

https://roda.sentinel-hub.com/sentinel-s2-l1c/tiles/17/S/ND/2023/3/17/0/preview.jpg


In [12]:
import rioxarray
b01_href = assets["B01"].href
b01 = rioxarray.open_rasterio(b01_href)
print(b01)

<xarray.DataArray (band: 1, y: 1830, x: 1830)>
[3348900 values with dtype=uint16]
Coordinates:
  * band         (band) int32 1
  * x            (x) float64 5e+05 5.001e+05 5.001e+05 ... 6.097e+05 6.098e+05
  * y            (y) float64 4.4e+06 4.4e+06 4.4e+06 ... 4.29e+06 4.29e+06
    spatial_ref  int32 0
Attributes:
    AREA_OR_POINT:       Area
    OVR_RESAMPLING_ALG:  AVERAGE
    _FillValue:          0
    scale_factor:        1.0
    add_offset:          0.0


In [19]:
items

id: S2B_17SND_20230317_0_L2A
"bbox: [-81.0002217816295, 38.75404518170041, -79.71873148679177, 39.7502589163948]"
datetime: 2023-03-17T16:22:31Z
platform: sentinel-2b
constellation: sentinel-2
instruments: ['msi']
gsd: 10
view:off_nadir: 0
proj:epsg: 32617
sentinel:utm_zone: 17
sentinel:latitude_band: S


In [18]:
## Let's see if we can stac_load the `items` variable
from odc.stac import stac_load

output_crs = CRS
zoom = 2**5

xx = stac_load(
    items,
    bands=("B04", "B03", "B02"),
    crs=crs,
    resolution=10
)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "rasterio\_warp.pyx", line 586, in rasterio._warp._reproject
  File "rasterio\_err.pyx", line 195, in rasterio._err.exc_wrap_int
rasterio._err.CPLE_AppDefinedError: /vsicurl/https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/17/S/ND/2023/1/S2A_17SND_20230131_0_L2A/B04.tif, band 1: IReadBlock failed at X offset 0, Y offset 0: TIFFReadEncodedTile() failed.

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\bbrown\Anaconda3\envs\stac_practice\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\bbrown\AppData\Local\Temp\ipykernel_33884\2666720200.py", line 7, in <module>
    xx = stac_load(
  File "c:\Users\bbrown\Anaconda3\envs\stac_practice\lib\site-packages\odc\stac\_load.py", line 610, in load
    for _ in _work:
  File "c:\Users\bbrown\Anaconda3\envs\stac_pract

In [ ]:
xx

NameError: name 'xx' is not defined